In [2]:
import pandas as pd

base_dir = "/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet"
df = pd.read_parquet(f'{base_dir}/partition_id=7/part-0.parquet')

In [3]:
# Sort the DataFrame
df = df.sort_values(by=['symbol_id', 'date_id', 'time_id'])
df = df.reset_index(drop=True)
df.head(100)


,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,1190,0,0,2.888326,0.928983,0.441290,1.352455,1.301461,2.302733,-0.231061,...,-0.203992,-0.257635,-0.254294,-0.100387,-0.390012,-1.553783,0.224214,-0.278927,-1.170454,0.546211
1,1190,1,0,2.888326,0.812045,0.287618,0.781931,0.862262,1.999738,-0.078465,...,-0.294544,-0.304375,-0.250175,0.242523,-0.562048,-1.679339,0.104258,-0.492223,-2.278074,-0.096518
2,1190,2,0,2.888326,1.564986,0.848806,0.845978,0.934356,2.146238,-0.020378,...,-0.194655,-0.269392,-0.376987,-0.162672,-1.052235,-1.176147,0.202343,-1.118208,-1.336344,0.471153
3,1190,3,0,2.888326,1.646784,0.556938,1.245883,1.384533,2.333363,-0.051726,...,-0.308909,-0.396191,-0.373662,-0.249862,-0.805066,-1.410163,0.085549,-0.550099,-1.818652,0.378067
4,1190,4,0,2.888326,1.048084,0.593710,0.952669,1.393819,1.513835,-0.096590,...,-0.297132,0.057213,-0.049167,0.489998,-0.336325,-1.074370,0.571558,-0.567549,-1.555979,0.374713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1190,95,0,2.888326,0.899313,0.872655,0.611904,0.800270,1.672438,0.091263,...,-0.238560,0.008676,-0.086372,0.032918,-0.112259,-0.088120,-0.338452,-0.069689,0.000172,-0.464786
96,1190,96,0,2.888326,1.152360,0.658832,0.786550,0.733082,1.936692,-0.055169,...,-0.291433,-0.016981,-0.095687,-0.091012,-0.149680,-0.272296,-0.294019,-0.130946,-0.178600,-0.216201
97,1190,97,0,2.888326,0.951265,0.968181,1.005434,0.923703,1.711512,-0.090984,...,-0.235122,-0.006187,-0.098262,-0.108648,-0.178640,-0.211416,-0.172619,-0.174573,-0.113896,-0.192201
98,1190,98,0,2.888326,0.944483,1.020565,0.983124,0.972003,1.554945,-0.003110,...,-0.336540,-0.022539,-0.096584,0.087890,0.077090,-0.070816,0.578915,0.150257,0.004600,0.944588


In [5]:
processed_groups = pd.DataFrame(columns = ['Symbol ID', 'Dropped Columns', 'Filled Columns', '% Filled'])

percentage_threshold = 10.0

for symbol_id, group in df.groupby('symbol_id'):

    # print(f"\nSymbol ID: {symbol_id}")
    # processed_groups['Symbol ID'].append(symbol_id)

    dropped_columns = []
    filled_columns = []
    percent_filled = []

    for feature in group.columns:
        total_rows = len(group)
        empty_rows = group[feature].isnull().sum()
        empty_rows_percentage = (empty_rows / total_rows) * 100
        nonempty_rows_percentage = 100 - empty_rows_percentage
    
        # print(f'{feature} : empty row = {empty_rows_percentage:.2f}% - non empty rows = {nonempty_rows_percentage:.2f}%')  

        if empty_rows == total_rows or empty_rows_percentage >= percentage_threshold:
            df = df.drop(columns=feature)
            dropped_columns.append(feature)
            
        elif empty_rows > 0:
            # Forward-fill and backward-fill missing values
            df.loc[group.index,feature] = group[feature].ffill().bfill()
            filled_columns.append(feature)
            percent_filled.append(empty_rows_percentage)

    # print(f"Symbol ID: {symbol_id}")
    # print(f"Dropped Columns: {dropped_columns}")
    # print(f"Filled Columns: {filled_columns}")
    # print(f"% Filled: {percent_filled}")



    # Check if there is data to add to the DataFrame
    
    if dropped_columns or filled_columns or percent_filled:
        # new row for table
        new_row = pd.DataFrame({
            'Symbol ID': [symbol_id],
            'Dropped Columns': [dropped_columns],
            'Filled Columns': [filled_columns],
            '% Filled': [percent_filled]
        })

    
    processed_groups = pd.concat([processed_groups, new_row], ignore_index=True)



print(processed_groups)
    




        

   Symbol ID Dropped Columns  \
0         38              []   
1         38              []   
2         38              []   
3         38              []   
4         38              []   
5         38              []   
6         38              []   
7         38              []   
8         38              []   
9         38              []   
10        38              []   
11        38              []   
12        38              []   
13        38              []   
14        38              []   
15        38              []   
16        38              []   
17        38              []   
18        38              []   
19        38              []   
20        38              []   
21        38              []   
22        38              []   
23        38              []   
24        38              []   
25        38              []   
26        38              []   
27        38              []   
28        38              []   
29        38              []   
30      

In [45]:
# this is just a double check

null_columns = df.columns[df.isnull().any()]

print(null_columns)

Index([], dtype='object')


In [46]:
df.columns

Index(['date_id', 'time_id', 'symbol_id', 'weight', 'feature_00', 'feature_01',
       'feature_02', 'feature_03', 'feature_04', 'feature_05', 'feature_06',
       'feature_07', 'feature_08', 'feature_09', 'feature_10', 'feature_11',
       'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16',
       'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_22',
       'feature_23', 'feature_24', 'feature_25', 'feature_28', 'feature_29',
       'feature_30', 'feature_32', 'feature_33', 'feature_34', 'feature_35',
       'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40',
       'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45',
       'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50',
       'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55',
       'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60',
       'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_6

- `date_id` and `time_id` - Integer values that are ordinally sorted, providing a chronological structure to the data, although the **actual time intervals** between `time_id` values may vary.
- `symbol_id` - Identifies a unique financial instrument. Each row in the dataset corresponds to a unique combination of a symbol (identified by `symbol_id`) and a timestamp (represented by `date_id` and `time_id`). The `symbol_id` column contains encrypted identifiers. Each `symbol_id` is not guaranteed to appear in all `time_id` and `date_id` combinations.
- `weight` - The weighting used for calculating the scoring function.
- `feature_{00...78}` - Anonymized market data.
- `responder_{0...8}` - Anonymized responders clipped between -5 and 5. The responder_6 field is what you are trying to predict.

In [4]:
TEMPORAL_FEATURES = ['date_id', 'time_id','symbol_id']
MARKET_FEATURES = [f'feature_{i:02}' for i in range(0,79)]
RESPONDER_FEATURES = [f'responder_{i}' for i in range(0,9)]
RESPONDER_FEATURES.remove('responder_6')
SYMBOL_FEATURES = ['symbol_id']
WEIGHT = ['WEIGHT']

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1944210 entries, 0 to 1944209
Data columns (total 92 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date_id      int16  
 1   time_id      int16  
 2   symbol_id    int8   
 3   weight       float32
 4   feature_00   float32
 5   feature_01   float32
 6   feature_02   float32
 7   feature_03   float32
 8   feature_04   float32
 9   feature_05   float32
 10  feature_06   float32
 11  feature_07   float32
 12  feature_08   float32
 13  feature_09   int8   
 14  feature_10   int8   
 15  feature_11   int16  
 16  feature_12   float32
 17  feature_13   float32
 18  feature_14   float32
 19  feature_15   float32
 20  feature_16   float32
 21  feature_17   float32
 22  feature_18   float32
 23  feature_19   float32
 24  feature_20   float32
 25  feature_21   float32
 26  feature_22   float32
 27  feature_23   float32
 28  feature_24   float32
 29  feature_25   float32
 30  feature_26   float32
 31  feature_27   float32
 32

In [47]:
df.head(100)

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,1190,0,0,2.888326,0.928983,0.441290,1.352455,1.301461,2.302733,-0.231061,...,-0.203992,-0.257635,-0.254294,-0.100387,-0.390012,-1.553783,0.224214,-0.278927,-1.170454,0.546211
1,1190,1,0,2.888326,0.812045,0.287618,0.781931,0.862262,1.999738,-0.078465,...,-0.294544,-0.304375,-0.250175,0.242523,-0.562048,-1.679339,0.104258,-0.492223,-2.278074,-0.096518
2,1190,2,0,2.888326,1.564986,0.848806,0.845978,0.934356,2.146238,-0.020378,...,-0.194655,-0.269392,-0.376987,-0.162672,-1.052235,-1.176147,0.202343,-1.118208,-1.336344,0.471153
3,1190,3,0,2.888326,1.646784,0.556938,1.245883,1.384533,2.333363,-0.051726,...,-0.308909,-0.396191,-0.373662,-0.249862,-0.805066,-1.410163,0.085549,-0.550099,-1.818652,0.378067
4,1190,4,0,2.888326,1.048084,0.593710,0.952669,1.393819,1.513835,-0.096590,...,-0.297132,0.057213,-0.049167,0.489998,-0.336325,-1.074370,0.571558,-0.567549,-1.555979,0.374713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1190,95,0,2.888326,0.899313,0.872655,0.611904,0.800270,1.672438,0.091263,...,-0.238560,0.008676,-0.086372,0.032918,-0.112259,-0.088120,-0.338452,-0.069689,0.000172,-0.464786
96,1190,96,0,2.888326,1.152360,0.658832,0.786550,0.733082,1.936692,-0.055169,...,-0.291433,-0.016981,-0.095687,-0.091012,-0.149680,-0.272296,-0.294019,-0.130946,-0.178600,-0.216201
97,1190,97,0,2.888326,0.951265,0.968181,1.005434,0.923703,1.711512,-0.090984,...,-0.235122,-0.006187,-0.098262,-0.108648,-0.178640,-0.211416,-0.172619,-0.174573,-0.113896,-0.192201
98,1190,98,0,2.888326,0.944483,1.020565,0.983124,0.972003,1.554945,-0.003110,...,-0.336540,-0.022539,-0.096584,0.087890,0.077090,-0.070816,0.578915,0.150257,0.004600,0.944588


In [6]:
# Normalize Features

from sklearn.preprocessing import StandardScaler

df_feature_cols = []
possible_feature_cols = [f'feature_{i:02}' for i in range(0,79)]

for feature in df.columns:
    if feature in possible_feature_cols:
        df_feature_cols.append(feature)

std_scaler = StandardScaler()
df[df_feature_cols] = std_scaler.fit_transform(df[df_feature_cols])

In [52]:
df.head(100)

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,1190,0,0,2.888326,-0.109961,0.348639,0.219882,0.180069,2.025294,-0.102465,...,-0.218812,-0.257635,-0.254294,-0.100387,-0.390012,-1.553783,0.224214,-0.278927,-1.170454,0.546211
1,1190,1,0,2.888326,-0.201104,0.222629,-0.227093,-0.163766,1.758053,0.038524,...,-0.335070,-0.304375,-0.250175,0.242523,-0.562048,-1.679339,0.104258,-0.492223,-2.278074,-0.096518
2,1190,2,0,2.888326,0.385749,0.682802,-0.176916,-0.107326,1.887265,0.092192,...,-0.206824,-0.269392,-0.376987,-0.162672,-1.052235,-1.176147,0.202343,-1.118208,-1.336344,0.471153
3,1190,3,0,2.888326,0.449504,0.443471,0.136389,0.245104,2.052309,0.063229,...,-0.353514,-0.396191,-0.373662,-0.249862,-0.805066,-1.410163,0.085549,-0.550099,-1.818652,0.378067
4,1190,4,0,2.888326,-0.017132,0.473624,-0.093329,0.252373,1.329487,0.021778,...,-0.338394,0.057213,-0.049167,0.489998,-0.336325,-1.074370,0.571558,-0.567549,-1.555979,0.374713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1190,95,0,2.888326,-0.133087,0.702359,-0.360300,-0.212297,1.469375,0.195341,...,-0.263193,0.008676,-0.086372,0.032918,-0.112259,-0.088120,-0.338452,-0.069689,0.000172,-0.464786
96,1190,96,0,2.888326,0.064142,0.527023,-0.223475,-0.264897,1.702447,0.060048,...,-0.331077,-0.016981,-0.095687,-0.091012,-0.149680,-0.272296,-0.294019,-0.130946,-0.178600,-0.216201
97,1190,97,0,2.888326,-0.092594,0.780690,-0.051990,-0.115666,1.503838,0.026957,...,-0.258780,-0.006187,-0.098262,-0.108648,-0.178640,-0.211416,-0.172619,-0.174573,-0.113896,-0.192201
98,1190,98,0,2.888326,-0.097880,0.823644,-0.069469,-0.077853,1.365746,0.108147,...,-0.388988,-0.022539,-0.096584,0.087890,0.077090,-0.070816,0.578915,0.150257,0.004600,0.944588


In [7]:
X = df.drop(columns = ['responder_1', 'responder_2', 'responder_3', 'responder_4', 'responder_5', 'responder_6', 'responder_7', 'responder_8', 'weight'])
y = df['responder_6']




In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Gaussian Naive Bayes

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X_train)

poly_reg = LinearRegression()
poly_reg.fit(X_poly, y_train)

poly_y_pred = poly_reg.predict(X_test)

poly_mse = mean_squared_error(y_test,y_pred)
print(f'Polynomial MSE = {gnb_mse}')

## Linear Regression with Ridge, Lasso, and Elastic

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

ridge_mse = []
# 100 alpha values from 0.001 to 1000
alpha_counter = np.logspace(-3, 3, 100)  
# alpha_counter = np.arange(0.1,1000,0.1)

for a in alpha_counter:
    # Ridge Regression
    ridge = Ridge(alpha=a)
    ridge.fit(X_train, y_train)
    ridge_y_pred = ridge.predict(X_test)
    ridge_mse.append(mean_squared_error(y_test, ridge_y_pred))

plt.plot(alpha_counter,ridge_mse)

# argmin() finds index of minimnum value
min_mse_index = np.argmin(etaMSE)

print(f'Minimum MSE = {ridge_mse[min_mse_index]} at alpha = {alpha_counter[min_mse_index]}')

In [ ]:
# Lasso Regression
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
lasso_y_pred = lasso.predict(X_test)
lasso_mse = mean_squared_error(y_test, lasso_y_pred)
print(f"Lasso Regression MSE: {lasso_mse}")

# Elastic Net Regression --> The combination of Ridge and Lasso
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(X_train, y_train)
elastic_net_y_pred = elastic_net.predict(X_test)
elastic_net_mse = mean_squared_error(y_test, elastic_net_y_pred)
print(f"Elastic Net Regression MSE: {elastic_net_mse}")

THIS IS my CRAP FROM HERE ON OUT 

In [5]:
# empty_columns = []
# fully_filled_columns = []
# partially_empty_columns = []
# for feature in list(df.columns):
#     # empty_rows = df_pl[feature].is_null().sum()
#     # nonempty_rows = len(df_pl[feature]) - empty_rows
    
#     empty_rows_percentage = (df[feature].isnull().sum() / len(df))
#     nonempty_rows_percentage = 1.0 - empty_rows_percentage

#     print(f'{feature} : empty row = {empty_rows_percentage*100}% - non empty rows = {nonempty_rows_percentage*100}%')                                                                                 
                                                                                     

#     # ## store in feature_stats since recalculating empty rows for all 79 features takes a while
#     # if nonempty_rows == 0:
#     #     empty_columns.append(feature)

#     # elif empty_rows == 0:
#     #     fully_filled_columns.append(feature)

#     # else:
#     #     partially_empty_columns.append(feature)
    
#     # print(f'{feature} : total  - {len(df[[feature]])} - empty - {empty_rows} - nonempty - {nonempty_rows}')

date_id : empty row = 0.0% - non empty rows = 100.0%
time_id : empty row = 0.0% - non empty rows = 100.0%
symbol_id : empty row = 0.0% - non empty rows = 100.0%
weight : empty row = 0.0% - non empty rows = 100.0%
feature_00 : empty row = 0.0% - non empty rows = 100.0%
feature_01 : empty row = 0.0% - non empty rows = 100.0%
feature_02 : empty row = 0.0% - non empty rows = 100.0%
feature_03 : empty row = 0.0% - non empty rows = 100.0%
feature_04 : empty row = 0.0% - non empty rows = 100.0%
feature_05 : empty row = 0.0% - non empty rows = 100.0%
feature_06 : empty row = 0.0% - non empty rows = 100.0%
feature_07 : empty row = 0.0% - non empty rows = 100.0%
feature_08 : empty row = 1.1917494270435447% - non empty rows = 98.80825057295645%
feature_09 : empty row = 0.0% - non empty rows = 100.0%
feature_10 : empty row = 0.0% - non empty rows = 100.0%
feature_11 : empty row = 0.0% - non empty rows = 100.0%
feature_12 : empty row = 0.0% - non empty rows = 100.0%
feature_13 : empty row = 0.0% - 

In [37]:
# Sort by symbol id and by date/time:


# # Sort the DataFrame
# df = df.sort_values(by=['symbol_id', 'date_id', 'time_id'])

# # Group by 'symbol_id' and apply both ffill and bfill
# df = df.groupby('symbol_id').apply(lambda group: group.ffill().bfill()).reset_index(drop=True)



In [44]:
df.head(20)
null_columns = df.columns[df.isnull().any()]

print(null_columns)

Index([], dtype='object')


So we have 3 cases
1. Fully Empty columns : we can just drop them
2. Partially Empty Columns: we can impute them
3. Full Columns : No imputation necessary

In [21]:
df.sort_values(['time_id','date_id'])

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,1360,0,0,1.865621,2.891473,0.353332,1.668053,2.853531,3.412320,-1.245219,...,-0.352222,0.502829,0.115287,-0.313245,0.651638,-1.308276,-1.620408,0.374767,-1.049378,-2.496996
1,1360,0,1,1.823830,3.157964,0.191611,2.494816,2.845078,3.090568,-0.742658,...,-0.212594,0.351176,-0.131981,0.127530,0.053275,-0.241975,-0.027899,-0.184599,-0.216430,-0.182844
2,1360,0,2,1.411181,2.716050,0.036010,1.966540,2.863158,2.959575,-1.340717,...,3.219775,-0.883570,-0.445202,0.102318,-0.391579,-0.871908,0.363652,0.236567,-1.445096,0.506134
3,1360,0,3,1.544636,2.704180,0.410197,2.214327,3.090716,3.155712,-0.923788,...,0.281109,-0.524510,-0.459967,-0.971227,-0.524694,0.761837,-1.442856,-0.447208,0.857494,-1.468352
4,1360,0,4,1.061685,2.375254,0.886124,2.317989,3.283024,2.975022,-1.422651,...,0.217880,-0.623650,-0.370462,0.063419,-0.597050,-0.952827,-3.025858,-0.506223,-0.317776,-4.478455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6140019,1529,967,34,2.606971,0.722274,-1.008289,0.868799,0.916192,0.610800,0.698455,...,-0.077911,0.262104,0.115894,-0.333845,-0.020673,-0.010249,0.016540,-0.063972,0.002923,-0.120635
6140020,1529,967,35,1.311619,0.534866,-0.875589,0.983309,0.621512,0.201628,0.664715,...,-0.178959,1.510625,1.336293,-0.561086,0.129558,0.050797,0.001942,-0.230132,-0.073900,-0.338427
6140021,1529,967,36,1.496826,0.108323,-1.098514,1.064766,0.649984,0.259670,0.481629,...,-0.104596,0.289079,-0.016187,0.683824,0.160069,0.081944,0.230783,-0.207259,-0.048885,-0.400022
6140022,1529,967,37,1.407042,0.228048,-0.850542,0.429382,0.660879,-0.148156,0.916373,...,-0.831250,0.410921,0.259873,-0.158685,-0.033464,-0.019048,-0.052862,-0.082444,-0.007100,-0.148950


In [23]:
df[df.date_id==0].symbol_id.unique()

array([], dtype=int8)

In [24]:
df[df.date_id==1].symbol_id.unique()

array([], dtype=int8)

In [25]:
df[df.symbol_id == 1].date_id.unique()

array([1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370,
       1371, 1372, 1373, 1375, 1376, 1377, 1378, 1379, 1381, 1382, 1383,
       1384, 1385, 1386, 1388, 1389, 1395, 1396, 1397, 1398, 1399, 1400,
       1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411,
       1412, 1414, 1415, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423,
       1424, 1425, 1426, 1427, 1428, 1429, 1430, 1431, 1432, 1433, 1434,
       1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445,
       1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1455, 1457,
       1458, 1459, 1460, 1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468,
       1469, 1470, 1471, 1472, 1473, 1474, 1475, 1476, 1477, 1479, 1480,
       1481, 1483, 1484, 1485, 1486, 1487, 1488, 1489, 1490, 1491, 1492,
       1493, 1494, 1495, 1496, 1497, 1498, 1499, 1500, 1501, 1503, 1504,
       1505, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514, 1515,
       1516, 1517, 1518, 1519, 1520, 1521, 1522, 15

In [26]:
set(df[df.symbol_id == 1].date_id.unique()) - set(df[df.symbol_id == 7].date_id.unique()) 

{1389, 1433, 1486, 1500}

In [27]:
mask1 = (df.symbol_id == 1) & (df.date_id == 0)
mask2 = (df.symbol_id == 7) & (df.date_id == 0)
set(df[mask1].time_id.unique()) - set(df[mask2].date_id.unique()) 

set()

## Splitting Temporally

In [28]:
df = df.sort_values(['date_id', 'time_id'])
date_counts = df.date_id.value_counts()

In [30]:
date_counts = pd.DataFrame(date_counts.sort_index())
date_counts['cumulative_sum'] = date_counts['count'].cumsum()
date_counts.head()

,count,cumulative_sum
date_id,,
1360,37752,37752
1361,37752,75504
1362,37752,113256
1363,37752,151008
1364,36784,187792


We want to do a 80-20 split based on date

In [35]:
total = len(df)
split = 0.8
apprx_train_len = int(total*split)
apprx_test_len = total - apprx_train_len
date_counts['Train'] = date_counts['cumulative_sum'] < apprx_train_len
date_counts.head()

,count,cumulative_sum,Train
date_id,,,
0,6792,6792,True
1,10188,16980,True
2,9339,26319,True
3,10188,36507,True
4,10188,46695,True


In [41]:
last_train_data = date_counts[date_counts.Train == True].tail(1) 
last_train_data

,count,cumulative_sum,Train
date_id,,,
142,16131,1546029,True


In [42]:
first_test_data = date_counts[date_counts.Train == False].head(1)
first_test_data

,count,cumulative_sum,Train
date_id,,,
143,15282,1561311,False


In [51]:
train_len = last_train_data['cumulative_sum'].values[0]
test_len = len(df) - last_train_data['cumulative_sum'].values[0]
print("Training data :", train_len,"-" ,round(train_len*100/len(df),2), "%" )
print("Testing data :", test_len, "-", round(test_len*100/len(df),2), "%")

Training data : 1546029 - 79.52 %
Testing data : 398181 - 20.48 %


In [60]:
df.loc[:, 'Train'] = df.date_id <= last_train_data.index[0]
df.head()

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,Split,Train
0,0,0,1,3.889038,NaN,NaN,NaN,NaN,NaN,0.851033,...,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504,True,True
1,0,0,7,1.370613,NaN,NaN,NaN,NaN,NaN,0.676961,...,1.190077,-0.523998,3.849921,2.626981,5.000000,0.703665,0.216683,0.778639,True,True
2,0,0,9,2.285698,NaN,NaN,NaN,NaN,NaN,1.056285,...,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828,True,True
3,0,0,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.139366,...,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516,True,True
4,0,0,14,0.440570,NaN,NaN,NaN,NaN,NaN,0.955200,...,-0.502764,-0.348021,-3.928148,-1.591366,-5.000000,-3.572820,-1.089123,-5.000000,True,True


In [ ]:
df = df.drop_columns(empty_columns)
train_df = df[df.Train == True]
test_df = df[df.Train == False]
# train_df.to_csv('train.csv', index = False)
# test_df.to_csv('test.csv', index = False)

In [ ]:
# %load_ext cudf.pandas
# import pandas as pdd